In [1]:
import os
import glob
import pandas as pd
from datetime import datetime

directory = 'raw'
prefix = 'mm'
ts_column = 'ts'

pattern = os.path.join(directory, f"{prefix}*.csv")
files = glob.glob(pattern)

all_dates = []

for file in files:
    try:
        df = pd.read_csv(file, sep=';', usecols=[ts_column])
        dates = pd.to_datetime(df[ts_column], errors='coerce').dropna()
        all_dates.extend(dates)
    except Exception as e:
        print(f"⚠️ Failed to read {os.path.basename(file)}: {e}")

if all_dates:
    min_date = min(all_dates)
    max_date = max(all_dates)
    print(f"📅 Oldest date across all files: {min_date.date()}")
    print(f"📅 Newest date across all files: {max_date.date()}")
else:
    print("❌ No valid 'ts' dates found in any mm*.csv file.")


📅 Oldest date across all files: 2015-08-12
📅 Newest date across all files: 2020-05-28


In [2]:
import requests
import pandas as pd
from datetime import datetime

def fetch_open_meteo(city_name, lat, lon, start_date, end_date):
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&hourly=temperature_2m,precipitation,surface_pressure,snowfall,"
        f"shortwave_radiation,winddirection_10m,windgusts_10m,windspeed_10m,"
        f"relative_humidity_2m&timezone=Europe%2FBerlin"
    )
    print(f"Fetching: {city_name}")
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data for {city_name}: {response.text}")

    data = response.json()["hourly"]
    df = pd.DataFrame(data)
    df["datetime"] = pd.to_datetime(df["time"])
    df["city"] = city_name
    return df.drop(columns=["time"])

# Setup
start = min_date.date()
end = max_date.date()

cities = {
    "Ljubljana": (46.0569, 14.5058),
    "Maribor": (46.5547, 15.6459),
    "Celje": (46.2309, 15.2604),
    "Koper": (45.5481, 13.7302),
    "Novo Mesto": (45.8030, 15.1688)
}

# Fetch and concatenate
dfs = [fetch_open_meteo(city, lat, lon, start, end) for city, (lat, lon) in cities.items()]
all_data = pd.concat(dfs)

# Aggregate to country level (mean)
country_hourly = (
    all_data
    .drop(columns=["city"])
    .groupby("datetime")
    .mean(numeric_only=True)
    .round(4)
    .reset_index()
)


# Save
country_hourly.to_csv("processed/slovenia_averaged_weather.csv", index=False)
print("Saved to processed/slovenia_averaged_weather.csv")

Fetching: Ljubljana
Fetching: Maribor
Fetching: Celje
Fetching: Koper
Fetching: Novo Mesto
Saved to processed/slovenia_averaged_weather.csv


In [4]:
# Extract years from the date range
start_year = start.year
end_year = end.year

dates = []
for year in range(start_year, end_year + 1):
    url = f"https://date.nager.at/api/v3/publicholidays/{year}/SI"
    try:
        response = requests.get(url)
        response.raise_for_status()
        dates += [h["date"] for h in response.json()]
    except Exception as e:
        print(f"Failed for {year}: {e}")

# Deduplicate, sort, and save
df = pd.DataFrame(sorted(set(dates)), columns=["holiday"])
df.to_csv("processed/slovenia_holidays.csv", index=False)
print("Saved to processed/slovenia_holidays.csv")

Saved to processed/slovenia_holidays.csv


In [2]:
import os
import pandas as pd
from glob import glob

raw_dir = 'raw'
out_dir = 'processed'
os.makedirs(out_dir, exist_ok=True)

files = glob(os.path.join(raw_dir, '*.csv'))

for file in files:
    filename = os.path.basename(file)
    try:
        df = pd.read_csv(file, sep=';', usecols=['ts', 'vrednost'])

        # Convert time and vrednost
        df['ts'] = pd.to_datetime(df['ts'], errors='coerce')
        df['vrednost'] = df['vrednost'].str.replace(',', '.', regex=False).astype(float)

        # Group by date and compute average
        df['date'] = df['ts'].dt.date
        daily_avg = df.groupby('date', as_index=False)['vrednost'].mean()

        # Sort by date
        daily_avg = daily_avg.sort_values(by='date')

        # Save to processed folder
        out_path = os.path.join(out_dir, filename)
        daily_avg.to_csv(out_path, index=False)
        print(f"✅ Processed: {filename}")
    except Exception as e:
        print(f"❌ Failed: {filename} — {e}")


✅ Processed: mm254.csv
✅ Processed: mm3371.csv
✅ Processed: mm4678.csv
✅ Processed: mm9725.csv
✅ Processed: mm41865.csv
✅ Processed: mm45674.csv
✅ Processed: mm57710.csv
✅ Processed: mm65924.csv
✅ Processed: mm72425.csv
✅ Processed: mm79158.csv
✅ Processed: mm118648.csv
✅ Processed: mm125431.csv
✅ Processed: mm144644.csv
✅ Processed: mm148296.csv
✅ Processed: mm166942.csv
✅ Processed: mm182972.csv
✅ Processed: mm187195.csv
✅ Processed: mm199762.csv
